In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import math
import logging
sys.path.append('../')

#import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb
#from process.footprint_report import update_ln, update_poly, update_pt
from shapely import Polygon, MultiPolygon
from shapely import affinity
#import dask_geopandas

from its_logging.logger_config import logger
logger = logging.getLogger('process.footprint')

In [2]:
upload_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload"

In [ ]:
activity_gdf = gpd.read_file(os.path.join(upload_path, 'ITS_V2_0_Activity_webmap.gdb'),
                            driver='OpenFileGDB',
                            layer='ITSV2_0_Activity')

In [9]:
footprint_gdf = gpd.read_file(os.path.join(upload_path, 'ITS_V2_0_Footprint_Polygons.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Footprint_Polygons_gdb')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [3]:
line_gdf = gpd.read_file(os.path.join(upload_path, 'ITS_V2_0_Lines.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Lines')
point_gdf = gpd.read_file(os.path.join(upload_path, 'ITS_V2_0_Points.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Points')
poly_gdf = gpd.read_file(os.path.join(upload_path, 'ITS_V2_0_Polygons.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Polygons')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


In [21]:
from datetime import datetime

In [28]:
def get_activity_report(enriched_points, enriched_lines, enriched_polygons):
    append_all = pd.concat([enriched_lines, enriched_points, enriched_polygons])
    append_all = append_all[(append_all['COUNTS_TO_MAS'] == 'YES')]
    
    # 
    append_all.geometry = append_all.representative_point()
    
    
    append_all = append_all[["AGENCY",
        "ADMINISTERING_ORG",
        "PRIMARY_OWNERSHIP_GROUP",
        "COUNTY",
        "REGION",
        "ACTIVITY_DESCRIPTION",
        "ACTIVITY_CAT",
        "BROAD_VEGETATION_TYPE",
        "ACTIVITY_STATUS",
        "ACTIVITY_QUANTITY",
        "ACTIVITY_UOM",
        "ACTIVITY_END",
        "Year_txt",
        "geometry"
    ]]
    
    # check if geometry is_valid
    # personally this is redundent, unless lat, lon is not valid, but that would throw an error in previous part
    append_all = append_all[append_all.is_valid]
    
    
    def get_entity_type(agency):
        if agency in ['CALEPA', 'CALSTA', 'CNRA', 'PARKS', 'California State Parks']:
            return 'State'
        if agency in ['DOD', 'DOI', 'USDA', 'DOE', 'NPS']:
            return 'Federal'
        if agency in ['Industrial Timber', 'Timber Companies', 'TIMBER']:
            return 'Timber Companies'
        else:
            return None
        
        
    append_all['ENTITY_TYPE'] = append_all['AGENCY'].apply(get_entity_type)
    
    return append_all

In [29]:
activity_report_gdf = get_activity_report(point_gdf, line_gdf, poly_gdf)

In [30]:
activity_report_gdf.ACTIVITY_STATUS.unique()

array(['COMPLETE', 'ACTIVE'], dtype=object)

In [31]:
activity_report_gdf[activity_report_gdf.ACTIVITY_CAT.isna()].AGENCY.unique()

array([], dtype=object)

In [32]:
activity_report_gdf[(activity_report_gdf.ACTIVITY_DESCRIPTION == "AMW_AREA_RESTOR") | (activity_report_gdf.ACTIVITY_DESCRIPTION == "OAK_WDLND_MGMT")].ACTIVITY_CAT.unique()

array(['MECH_HFR'], dtype=object)

In [36]:
activity_report_gdf = activity_report_gdf[(activity_report_gdf.Year_txt == '2021') |
                    (activity_report_gdf.Year_txt == '2022') |
                    (activity_report_gdf.Year_txt == '2023') |
                    (activity_report_gdf.Year_txt == '2024')]

In [38]:
activity_report_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0_Activity_webmap2.gdb'),
                            driver='OpenFileGDB',
                            layer='ITSV2_0_Activity')

2025-08-29 11:50:00,716 INFO  [pyogrio._io           ]  Created 160,989 records


In [14]:
def get_tp(gdf):
    gdf = gdf[gdf.COUNTS_TO_MAS == "YES"]
    gdf = gdf[gdf.Year_txt == '2023']
    return gdf[gdf.ACTIVITY_CAT == 'TREE_PLNTING']

In [15]:
tp_2023_gdf = pd.concat([get_tp(line_gdf), get_tp(poly_gdf), get_tp(point_gdf)])

In [4]:
f_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V1.1"
f_name = "Interagency_Tracking_System_V1.1_full_no_personal.gdb"

colname_sample_gdf = gpd.read_file(os.path.join(f_path, f_name), driver="OpenFileGDB", layer="ITS_V1_1_points")

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [5]:
import numpy as np

In [6]:
no_personal_cols = np.intersect1d(line_gdf.columns,colname_sample_gdf.columns)

In [7]:
dash_name= "Interagency_Tracking_System_V1.1_dashboard.gdb"

In [8]:

dashname_gdf = gpd.read_file(os.path.join(f_path, dash_name), driver="OpenFileGDB", layer="ITS_V1_1_points")

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [28]:
line_gdf = line_gdf[no_personal_cols]
point_gdf = point_gdf[no_personal_cols]
poly_gdf = poly_gdf[no_personal_cols]

In [29]:
line_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0_Lines.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Lines')
point_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0_Points.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Points')
poly_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0_Polygons.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Polygons')

2025-09-05 10:58:41,158 INFO  [pyogrio._io           ]  Created 131,658 records
2025-09-05 10:58:46,200 INFO  [pyogrio._io           ]  Created 20,443 records
2025-09-05 10:59:13,017 INFO  [pyogrio._io           ]  Created 454,597 records


In [44]:
line_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Lines')
point_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Points')
poly_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Polygons')
activity_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Activity')
footprint_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Footprint')

2025-09-05 11:28:33,713 INFO  [pyogrio._io           ]  Created 131,658 records
2025-09-05 11:28:38,734 INFO  [pyogrio._io           ]  Created 20,443 records
2025-09-05 11:29:05,114 INFO  [pyogrio._io           ]  Created 454,597 records
2025-09-05 11:29:14,610 INFO  [pyogrio._io           ]  Created 160,989 records
2025-09-05 11:29:17,932 INFO  [pyogrio._io           ]  Created 34,530 records


In [9]:
dash_cols = np.intersect1d(line_gdf.columns,dashname_gdf.columns)

In [10]:
dash_line_gdf = line_gdf[dash_cols][line_gdf.Year > 2020]
dash_point_gdf = point_gdf[dash_cols][point_gdf.Year > 2020]
dash_poly_gdf = poly_gdf[dash_cols][poly_gdf.Year > 2020]

In [12]:
dash_line_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0_Lines_webmap.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Lines')
dash_point_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0_Points_webmap.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Points')
dash_poly_gdf.to_file(os.path.join(upload_path, 'ITS_V2_0_Polygons_webmap.gdb'),
                            driver='OpenFileGDB',
                            layer='ITS_V2_0_Polygons')

2025-09-05 11:49:34,863 INFO  [pyogrio._io           ]  Created 103,807 records
2025-09-05 11:49:38,265 INFO  [pyogrio._io           ]  Created 11,174 records
2025-09-05 11:49:44,592 INFO  [pyogrio._io           ]  Created 69,308 records


In [15]:
len(line_gdf.columns)

97

In [26]:
line_gdf.ST_ABBREV

0         CA
1         CA
2         CA
3         CA
4         CA
          ..
131653    CA
131654    CA
131655    CA
131656    CA
131657    CA
Name: ST_ABBREV, Length: 131658, dtype: object

In [10]:
activity_gdf[activity_gdf.ADMINISTERING_ORG == 'FWS'].AGENCY.unique()

array(['DOI', 'FWS'], dtype=object)